In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import numpy as np 
import scipy
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import os # imports for loading data
from tqdm import tqdm

In [ ]:
# !pip install pystan==2.19.1.1

In [ ]:
# !pip install prophet

In [ ]:
# !pip install neuralprophet

In [ ]:
from prophet import Prophet
from neuralprophet import NeuralProphet

In [ ]:
from prophet.plot import plot_plotly
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode()
%matplotlib inline

In [ ]:
# Importing the signal in
df = pd.read_csv(os.path.abspath('turn_over.csv'), index_col=False) 
#df = pd.read_csv(os.path.abspath('/content/turn_over.csv'), index_col=False) 
print(f"Successfully read in {df.size} values.")
print("First 5 values:")
df.head(5)

In [ ]:
list_Month = [i.split(" ")[0][0:2] for i in  list(df['CALENDAR_DATE'])]
list_Date = [i.split(" ")[0][6:8] for i in  list(df['CALENDAR_DATE'])]
list_Year = [i.split(" ")[0][3:5] for i in  list(df['CALENDAR_DATE'])]

df['Month'] = list_Month
df['Date'] = list_Date
df['Year'] = list_Year

In [ ]:
df['ds'] = pd.DatetimeIndex(df['Year']+'-'+df['Month']+'-'+df['Date'])

In [ ]:
df= df.rename(columns=str.lower)

In [ ]:
df = df.drop_duplicates(inplace=False,keep='first')

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
df = df.groupby(['ds','cust_id','jurisdiction'])['turnover'].sum()
df = pd.DataFrame(df)
df = df.rename_axis(['ds', 'cust_id', 'jurisdiction']).reset_index()

In [ ]:
df['ds'].agg(['min', 'max'])
#334 days`

In [ ]:
df.cust_id.nunique()

In [ ]:
juris = list(set(df['jurisdiction']))
juris

In [ ]:
df1 = df[df['jurisdiction']=='DK'].reset_index(drop=True)
df2 = df[df['jurisdiction']=='BE'].reset_index(drop=True)
df3 = df[df['jurisdiction']=='MT'].reset_index(drop=True)
df4 = df[df['jurisdiction']=='IE'].reset_index(drop=True)
df5 = df[df['jurisdiction']=='DE'].reset_index(drop=True)
df6 = df[df['jurisdiction']=='NT'].reset_index(drop=True)
df7 = df[df['jurisdiction']=='SE'].reset_index(drop=True)
df8 = df[df['jurisdiction']=='EE'].reset_index(drop=True)
df9 = df[df['jurisdiction']=='RO'].reset_index(drop=True)
df10 = df[df['jurisdiction']=='IT'].reset_index(drop=True)
df11 = df[df['jurisdiction']=='UK'].reset_index(drop=True)


In [ ]:
df

In [ ]:
df['turnover'].sum()

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.turnover.sum()

In [ ]:
df1.cust_id.nunique()

In [ ]:
594*334

In [ ]:
# my_dates = pd.date_range('2021-07-01', '2022-05-30', freq='D')

# idx= pd.MultiIndex.from_product([my_dates, df1.cust_id.unique()],names=['ds', 'cust_id'])
# df1 = df1.set_index(['ds', 'cust_id']).reindex(idx).fillna(0).reset_index()

In [ ]:
# df1['jurisdiction'] = df1['jurisdiction'][0]

In [ ]:
def addzeros(dataframe):
    juris = dataframe['jurisdiction'][0]
    my_dates = pd.date_range('2021-07-01', '2022-05-30', freq='D')
    idx= pd.MultiIndex.from_product([my_dates, dataframe.cust_id.unique()],names=['ds', 'cust_id'])
    dataframe = dataframe.set_index(['ds', 'cust_id']).reindex(idx).fillna(0).reset_index()
    dataframe['jurisdiction'] = juris
    return (dataframe)

In [ ]:
df1 = addzeros(df1)
df2 = addzeros(df2)
df3 = addzeros(df3)
df4 = addzeros(df4)
df5 = addzeros(df5)
df6 = addzeros(df6)
df7 = addzeros(df7)
df8 = addzeros(df8)
df9 = addzeros(df9)
df10 = addzeros(df10)
df11 = addzeros(df11)

In [ ]:
dataframe = pd.concat([df1, df2,df3,df4,df5,df6,df7,df8,df9,df10,df11], ignore_index=True)

In [ ]:
dataframe

In [ ]:
#5520
customer = dataframe[dataframe["cust_id"] == 10081].reset_index()
#10081
#customer = df[df["cust_id"] == 5520].reset_index()


In [ ]:
customer.shape

In [ ]:
    c = customer.drop(['index','cust_id','jurisdiction'],axis=1)
    c.columns = ['ds', 'y']
    
    threshold_date = pd.to_datetime('2021-12-27')
    mask = c['ds'] < threshold_date
    # Split the data and select `ds` and `y` columns.
    df_train = c[mask][['ds', 'y']]
    df_test = c[~ mask][['ds', 'y']]

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
n = NeuralProphet(n_lags =2, num_hidden_layers =2,seasonality_mode = 'multiplicative',learning_rate = 0.01,
                  weekly_seasonality = True, daily_seasonality = True,collect_metrics = True)
n.add_seasonality(name='monthly', period=30.5, fourier_order=5)
modeln = n.fit(df_train, freq='D')

In [ ]:
futuren = n.make_future_dataframe(c,periods=150,n_historic_predictions=True)
forecastn = n.predict(futuren)

In [ ]:
mask2 = forecastn['ds'] < threshold_date

forecast_train = forecastn[mask2]
forecast_test = forecastn[~ mask2]

In [ ]:
import seaborn as sns
sns.set_style('darkgrid', {'axes.facecolor': '.9'})
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
forecastn = forecastn.fillna(value=np.nan)

In [ ]:
fig, ax = plt.subplots()

# ax.fill_between(
#     x=forecastn['ds'],
#     y1=forecastn['yhat_lower'],
#     y2=forecastn['yhat_upper'],
#     color=sns_c[2], 
#     alpha=0.25,
#     label=r'0.95 credible_interval'
# )
#all_numeric(forecastn.dropna())
sns.lineplot(x='ds', y='y', label='y_train', data=df_train, ax=ax)
sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
sns.lineplot(x='ds', y='yhat1', label='y_hat', data=forecastn, ax=ax)
ax.axvline(threshold_date, color=sns_c[3], linestyle='--', label='train test split')
ax.legend(loc='upper left')
ax.set(title='Dependent Variable', ylabel='');

In [ ]:
forecastn

In [ ]:
n.plot(forecastn)

In [ ]:
n.plot_components(forecastn)